In [1]:
# pip install git+https://github.com/vuthanhdatt/vnstock-data-python.git
# pip install python-binance
# pip install pandas_ta
# pip install plotly==5.5.0
# pip install yfinance

In [2]:
apikey = 'rz1iJcFceAwjl1CpFbM3axkx0fRDb6vhFQ2xZkqma2xCRysgKpH6v8qkRlMJX9R0'
secret = 'AMI2gLDPCSe84a9AIN2M9eD3rXkvgwvAOgSANBRG8Zd9HEWtr2Hknfyw4XArrhbZ'

In [3]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import date as dt

In [4]:
from vnstock_data.all_exchange import VnStock
COOKIES={"vts_usr_lg":"ABCDEF","language": "en-US","__RequestVerificationToken":"GhijKL"}
vndata = VnStock(COOKIES)

In [5]:
client = Client(apikey, secret)
tickers= client.get_all_tickers()
ticker_df= pd.DataFrame(tickers)
ticker_df.set_index('symbol',inplace=True)

# Getting Data Function

In [6]:
def get_crypto_data(symbol, interval, starttime):
    df = pd.DataFrame(client.get_historical_klines(symbol, interval, starttime))
    df.columns = ['Open Time','Open','High','Low','Close','Volume','Close Time','Quote Asset Volume','Number of Trades','TB Base Volume','TB Quote Volume','Ignore']
    df['Open Time'] = pd.to_datetime(df['Open Time']/1000, unit='s')
    df['Close Time'] = pd.to_datetime(df['Close Time']/1000, unit='s')
    numeric_columns= ['Open','High','Low','Close','Volume','Quote Asset Volume','TB Base Volume','TB Quote Volume']
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, axis=1)
    df= df[['Close Time','Open','High','Low','Close','Volume']]
    df['Diff']= df['Close'] - df['Open']
    df.loc[df['Diff'] >= 0, 'Color'] = 'gray'
    df.loc[df['Diff'] <= 0, 'Color'] = 'red'
    return df

In [7]:
def get_stock_data(symbol, interval, starttime):
    df=pd.DataFrame(yf.download(symbol,interval=interval,start=starttime,progress=False,auto_adjust=True)).dropna()
    df['Diff']= df['Close'] - df['Open']
    df.loc[df['Diff'] >= 0, 'Color'] = 'gray'
    df.loc[df['Diff'] <= 0, 'Color'] = 'red'
    return df

In [8]:
def get_vnstock_data(symbol, starttime, interval):
    df= vndata.price(symbol, starttime, dt.today()).dropna()
    df= df.reindex(index=df.index[::-1])
    if interval == '1W':
        df= df.resample('1W').mean()
    elif interval == '1M':
        df= df.resample('1M').mean()
    df['Adj Open']= (df['Open']*df['Adj Close'])/df['Close']
    df['Adj High']= (df['High']*df['Adj Close'])/df['Close']
    df['Adj Low']= (df['Low']*df['Adj Close'])/df['Close']
    df['Diff']= df['Close'] - df['Open']
    df.loc[df['Diff'] >= 0, 'Color'] = 'gray'
    df.loc[df['Diff'] <= 0, 'Color'] = 'red'
    return df

# Calculating Indicator Function

In [9]:
def get_crypto_indi(symbol, interval, starttime):
    df= get_crypto_data(symbol, interval, starttime)
    sma9 = ta.sma(df["Close"], length=9)
    sma20 = ta.sma(df["Close"], length=20)
    sma50 = ta.sma(df["Close"], length=50)
    sma200 = ta.sma(df["Close"], length=200)
    spt= ta.supertrend(high=df['High'],low=df['Low'],close=df['Close'],period=10,multiplier=3)
    macd= ta.macd(df["Close"], length=9)
    macd.loc[macd['MACDh_12_26_9'] >= 0, 'Color'] = 'gray'
    macd.loc[macd['MACDh_12_26_9'] <= 0, 'Color'] = 'red'
    return sma9,sma20,sma50,sma200,spt,macd

In [10]:
def get_stock_indi(symbol, interval, starttime):
    df= get_stock_data(symbol, interval, starttime)
    sma9 = ta.sma(df["Close"], length=9)
    sma20 = ta.sma(df["Close"], length=20)
    sma50 = ta.sma(df["Close"], length=50)
    sma200 = ta.sma(df["Close"], length=200)
    spt= ta.supertrend(high=df['High'],low=df['Low'],close=df['Close'],period=10,multiplier=3)
    macd= ta.macd(df["Close"], length=9)
    macd.loc[macd['MACDh_12_26_9'] >= 0, 'Color'] = 'gray'
    macd.loc[macd['MACDh_12_26_9'] <= 0, 'Color'] = 'red'
    return sma9,sma20,sma50,sma200,spt,macd

In [11]:
def get_vnstock_indi(symbol, starttime, interval):
    df= get_vnstock_data(symbol, starttime, interval)
    sma9 = ta.sma(df["Adj Close"], length=9)
    sma20 = ta.sma(df["Adj Close"], length=20)
    sma50 = ta.sma(df["Adj Close"], length=50)
    sma200 = ta.sma(df["Adj Close"], length=200)
    spt= ta.supertrend(high=df['Adj High'],low=df['Adj Low'],close=df['Adj Close'],period=10,multiplier=3)
    macd= ta.macd(df["Adj Close"], length=9)
    macd.loc[macd['MACDh_12_26_9'] >= 0, 'Color'] = 'gray'
    macd.loc[macd['MACDh_12_26_9'] <= 0, 'Color'] = 'red'
    return sma9,sma20,sma50,sma200,spt,macd

# Plot Chart Function

In [12]:
def plot_crypto_chart(symbol, interval, starttime):
    df=get_crypto_data(symbol,interval,starttime)
    # Plot candlesticks chart
    df_C = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.03,row_width=[0.7,0.2,0.7],subplot_titles=(symbol + ' '+ interval + ' with SMA, SuperTrend', 'Volume',symbol + ' ' + interval +' with MACD'))
    df_C.add_trace(go.Candlestick(x = df['Close Time'], open= df['Open'], high= df['High'], low= df['Low'], close= df['Close']),row = 1, col = 1)
    df_C.add_trace(go.Bar(x = df['Close Time'], y= df['Volume'],showlegend=False,marker={'color':df['Color']}),row = 2, col = 1)

    # Adding SMA
    df_C.add_trace(go.Scatter(x=df['Close Time'], y= get_crypto_indi(symbol, interval, starttime)[0], line = dict(color= '#ffffff'), name= 'SMA9'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df['Close Time'], y= get_crypto_indi(symbol, interval, starttime)[1], line = dict(color= '#fff176'), name= 'SMA20'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df['Close Time'], y= get_crypto_indi(symbol, interval, starttime)[2], line = dict(color= '#fbc02d'), name= 'SMA50'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df['Close Time'], y= get_crypto_indi(symbol, interval, starttime)[3], line = dict(color= '#f57f17'), name= 'SMA200'),row=1, col=1)
    
    # Adding Super Trend
    df_C.add_trace(go.Scatter(x=df['Close Time'], y= get_crypto_indi(symbol, interval, starttime)[4]['SUPERTs_7_3.0'], line = dict(color= '#f34f50'), name= 'Sell Super Trend'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df['Close Time'], y= get_crypto_indi(symbol, interval, starttime)[4]['SUPERTl_7_3.0'], line = dict(color= '#4caf50'), name= 'Buy Super Trend'),row=1, col=1)
    
    #MACD
    df_C.add_trace(go.Scatter(x=df['Close Time'], y= get_crypto_indi(symbol, interval, starttime)[5]['MACD_12_26_9'], line = dict(color= '#1f77b4'), name= 'MACD'),row=3, col=1)
    df_C.add_trace(go.Bar(x = df['Close Time'], y= get_crypto_indi(symbol, interval, starttime)[5]['MACDh_12_26_9'],showlegend=False,marker={'color':get_crypto_indi(symbol, interval, starttime)[5]['Color']}),row = 3, col = 1)
    df_C.add_trace(go.Scatter(x=df['Close Time'], y= get_crypto_indi(symbol, interval, starttime)[5]['MACDs_12_26_9'], line = dict(color= '#d62728'), name= 'MACD Signal'),row=3, col=1)

    # Update layout
    for i in range(3,0,-1):
        df_C.update_xaxes(row=i, col=1, rangeslider_visible=False)         # Remove Range Slider
    df_C.update_layout(template='plotly_dark')
    df_C.update_layout(bargap=0.2)
    df_C.update_layout(height=1000, width=1500)
    df_C.update_traces(name='Price', selector=dict(type='candlestick'))
    return df_C.show()

In [13]:
def plot_stock_chart(symbol, interval, starttime):
    df = get_stock_data(symbol,interval,starttime)
    # Plot candlesticks chart
    df_C = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.03,row_width=[0.7,0.2,0.7],subplot_titles=(symbol + ' '+ interval + ' with SMA, SuperTrend', 'Volume',symbol + ' ' + interval +' with MACD'))
    df_C.add_trace(go.Candlestick(x = df.index, open= df['Open'], high= df['High'], low= df['Low'], close= df['Close']),row = 1, col = 1)
    df_C.add_trace(go.Bar(x = df.index, y= df['Volume'],showlegend=False,marker={'color':df['Color']}),row = 2, col = 1)

    # Adding SMA
    df_C.add_trace(go.Scatter(x=df.index, y= get_stock_indi(symbol, interval, starttime)[0], line = dict(color= '#ffffff'), name= 'SMA9'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df.index, y= get_stock_indi(symbol, interval, starttime)[1], line = dict(color= '#fff176'), name= 'SMA20'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df.index, y= get_stock_indi(symbol, interval, starttime)[2], line = dict(color= '#fbc02d'), name= 'SMA50'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df.index, y= get_stock_indi(symbol, interval, starttime)[3], line = dict(color= '#f57f17'), name= 'SMA200'),row=1, col=1)
    
    # Adding Super Trend
    df_C.add_trace(go.Scatter(x=df.index, y= get_stock_indi(symbol, interval, starttime)[4]['SUPERTs_7_3.0'], line = dict(color= '#f34f50'), name= 'Sell Super Trend'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df.index, y= get_stock_indi(symbol, interval, starttime)[4]['SUPERTl_7_3.0'], line = dict(color= '#4caf50'), name= 'Buy Super Trend'),row=1, col=1)
    
    #MACD
    df_C.add_trace(go.Scatter(x=df.index, y= get_stock_indi(symbol, interval, starttime)[5]['MACD_12_26_9'], line = dict(color= '#1f77b4'), name= 'MACD'),row=3, col=1)
    df_C.add_trace(go.Bar(x = df.index, y= get_stock_indi(symbol, interval, starttime)[5]['MACDh_12_26_9'],showlegend=False,marker={'color':get_stock_indi(symbol, interval, starttime)[5]['Color']}),row = 3, col = 1)
    df_C.add_trace(go.Scatter(x=df.index, y= get_stock_indi(symbol, interval, starttime)[5]['MACDs_12_26_9'], line = dict(color= '#d62728'), name= 'MACD Signal'),row=3, col=1)

    # Update layout
    for i in range(3,0,-1):
        df_C.update_xaxes(row=i, col=1, rangeslider_visible=False)         # Remove Range Slider
    df_C.update_layout(template='plotly_dark')
    df_C.update_layout(height=1000, width=1500)
    df_C.update_traces(name='Price', selector=dict(type='candlestick'))
    return df_C.show()

In [14]:
def plot_vnstock_chart(symbol, starttime, interval):
    df = get_vnstock_data(symbol,starttime, interval)
    # Plot candlesticks chart
    df_C = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.03,row_width=[0.7,0.2,0.7],subplot_titles=(symbol+ ' ' + interval + ' with SMA, SuperTrend', 'Volume', symbol + ' ' + interval +' with MACD'))
    df_C.add_trace(go.Candlestick(x = df.index, open= df['Adj Open'], high= df['Adj High'], low= df['Adj Low'], close= df['Adj Close']),row = 1, col = 1)
    df_C.add_trace(go.Bar(x = df.index, y= df['Volume'],showlegend=False,marker={'color':df['Color']}),row = 2, col = 1)

    # Adding SMA
    df_C.add_trace(go.Scatter(x=df.index, y= get_vnstock_indi(symbol, starttime, interval)[0], line = dict(color= '#ffffff'), name= 'SMA9'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df.index, y= get_vnstock_indi(symbol, starttime, interval)[1], line = dict(color= '#fff176'), name= 'SMA20'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df.index, y= get_vnstock_indi(symbol, starttime, interval)[2], line = dict(color= '#fbc02d'), name= 'SMA50'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df.index, y= get_vnstock_indi(symbol, starttime, interval)[3], line = dict(color= '#f57f17'), name= 'SMA200'),row=1, col=1)
    
    # Adding Super Trend
    df_C.add_trace(go.Scatter(x=df.index, y= get_vnstock_indi(symbol, starttime, interval)[4]['SUPERTs_7_3.0'], line = dict(color= '#f34f50'), name= 'Sell Super Trend'),row=1, col=1)
    df_C.add_trace(go.Scatter(x=df.index, y= get_vnstock_indi(symbol, starttime, interval)[4]['SUPERTl_7_3.0'], line = dict(color= '#4caf50'), name= 'Buy Super Trend'),row=1, col=1)
    
    #MACD
    df_C.add_trace(go.Scatter(x=df.index, y= get_vnstock_indi(symbol, starttime, interval)[5]['MACD_12_26_9'], line = dict(color= '#1f77b4'), name= 'MACD'),row=3, col=1)
    df_C.add_trace(go.Bar(x = df.index, y= get_vnstock_indi(symbol, starttime, interval)[5]['MACDh_12_26_9'],showlegend=False,marker={'color':get_vnstock_indi(symbol, starttime, interval)[5]['Color']}),row = 3, col = 1)
    df_C.add_trace(go.Scatter(x=df.index, y= get_vnstock_indi(symbol, starttime, interval)[5]['MACDs_12_26_9'], line = dict(color= '#d62728'), name= 'MACD Signal'),row=3, col=1)

    # Update layout
    for i in range(3,0,-1):
        df_C.update_xaxes(row=i, col=1, rangeslider_visible=False)         # Remove Range Slider
    df_C.update_layout(template='plotly_dark')
    df_C.update_layout(height=1000, width=1500)
    df_C.update_traces(name='Price', selector=dict(type='candlestick'))
    return df_C.show()

# Final Function

In [15]:
def crypto(symbol, interval, starttime):
    get_crypto_data(symbol, interval, starttime)
    get_crypto_indi(symbol, interval, starttime)
    return plot_crypto_chart(symbol, interval, starttime)

In [16]:
def usstock(symbol, interval, starttime):
    get_stock_data(symbol, interval, starttime)
    get_stock_indi(symbol, interval, starttime)
    return plot_stock_chart(symbol, interval, starttime)

In [17]:
def vnstock(symbol, starttime, interval):
    get_vnstock_data(symbol, starttime, interval)
    get_vnstock_indi(symbol, starttime, interval)
    return plot_vnstock_chart(symbol, starttime, interval)

# Run function

In [18]:
crypto('BTCUSDT','1d','Jan 1 2018')

In [19]:
crypto('BTCUSDT','1w','Jan 1 2018')

In [20]:
crypto('BTCUSDT','1M','Jan 1 2018')

In [21]:
usstock('TSLA','1d','2015-1-1')

In [22]:
usstock('TSLA','1wk','2015-1-1')

In [23]:
usstock('TSLA','1mo','2015-1-1')

In [24]:
vnstock('VCB', '2020-1-1','1D')

In [25]:
vnstock('VCB','2020-1-1','1W')

In [26]:
vnstock('VCB','2015-1-1','1M')